In [30]:
from text_loader.loader import DataLoader
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import joblib
import mlflow
import datetime

### Pre-processing

In [8]:
data_loader = DataLoader(filepath='../data/Tweets.csv')
processed_features, labels = data_loader.preprocess()

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.3)

### Train & evaluate

In [34]:
mlflow.set_tracking_uri('file:///Users/meissanechami/MLEng-politicalparties-python/data') 
mlflow.set_experiment("MLflow ")
with mlflow.start_run():
        # model = XGBClassifier(booster='gbtree', max_depth=6, max_leaves=0)
        # model.fit(X_train, y_train)

        train_accuracy = accuracy_score(y_train, model.predict(X_train))
        accuracy = accuracy_score(y_test, model.predict(X_test))

        print("  train_accuracy: %s" % train_accuracy)
        print("  accuracy: %s" % accuracy)

        params = model.get_xgb_params()

        for param in params.keys():
            mlflow.log_param(param, params[param])

        mlflow.log_metric("train_acc", train_accuracy)
        mlflow.log_metric("test_acc", accuracy)

        # Log the model with a timestamp
        timestamp = datetime.datetime.now()
        model_name = f"model_{timestamp}"
        mlflow.sklearn.log_model(model, model_name)

2024/03/13 17:48:50 INFO mlflow.tracking.fluent: Experiment with name 'MLflow ' does not exist. Creating a new experiment.


  train_accuracy: 0.7417633257327914
  accuracy: 0.6977909711245615


/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/homebrew/Cellar/python@3.11/3.11.7_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
